In [20]:
import pdb
from math import log10, floor

import numpy as np
import scipy.optimize as op
import scipy.special as spec
from scipy import stats
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, MaxNLocator
import seaborn as sns
import corner

import astropy
from astropy import stats as astrostats

import emcee
import radvel

%matplotlib inline

In [21]:
sys_props   = pd.read_csv('legacy_tables/system_props_719.csv')
stellar     = pd.read_csv('legacy_tables/legacy_specmatch_medians.csv')
masses_only = pd.read_csv('legacy_tables/planet_list_719.csv')
trends      = pd.read_csv('legacy_tables/trend_list_719.csv')
data        = pd.read_csv('../rvdata/legacy_all_data.csv')

# Make table for planet parameters.

In [22]:
masses_real    = masses_only.query('status != "N" and status != "A"').reset_index(drop=True)
masses_good    = masses_real.query('mass != "nan"').reset_index(drop=True)#[masses_only.hostname != 'hip63510']
masses_binary  = masses_good.query('status == "S" or status == "SS"').reset_index(drop=True)
masses_planet  = masses_good.query('status != "S" and status != "SS"').reset_index(drop=True)
masses_old     = masses_planet.query('status == "K"').reset_index(drop=True)
masses_new     = masses_planet.query('status == "C" or status == "J"').reset_index(drop=True)
false_positive = masses_only.query('status == "N" or status == "A"')#[['hostname', 'k', 'per', 'status']]

In [23]:
masses_table = pd.DataFrame()
#masses_table = masses_table.sort_values('P').sort_values('Name').reset_index()
masses_table['Name'] = masses_planet['hostname']
masses_table['Msini (MJup)'] = masses_planet['mass_med']
masses_table['a (AU)'] = masses_planet['axis_med']
masses_table['P (d)'] = masses_planet['per_med']
masses_table['e'] = masses_planet['e_med']

names = np.unique(np.array(masses_table.Name))

nplanets = np.array(masses_table.groupby('Name').count()['e'])
db_nplanets = pd.DataFrame({'Name':names, 'nplanets':nplanets})
masses_table = pd.merge(masses_table, db_nplanets, on='Name').reset_index()

In [24]:
namedict = {0:' b', 1:' c', 2:' d', 3:' e', 4:' f', 5:' g', 6:' h', 7:' i'}

names = np.unique(np.array(masses_planet.hostname))
for name in names:
    masses_name = masses_table.query('Name == "{}"'.format(name))
    numbah = len(masses_name)
    pl_index = 0
    for index in np.arange(numbah) + masses_name.index[0]:
        masses_table.loc[index, 'Name'] += namedict[pl_index]  
        pl_index += 1
        
masses_planet['Name'] = masses_table['Name']

## Do the same for stellar binaries.

In [25]:
masses_table = pd.DataFrame()
#masses_table = masses_table.sort_values('P').sort_values('Name').reset_index()
masses_table['Name'] = masses_binary['hostname']
masses_table['Msini (MJup)'] = masses_binary['mass_med']
masses_table['a (AU)'] = masses_binary['axis_med']
masses_table['P (d)'] = masses_binary['per_med']
masses_table['e'] = masses_binary['e_med']

names = np.unique(np.array(masses_table.Name))

nbinarys = np.array(masses_table.groupby('Name').count()['e'])
db_nbinarys = pd.DataFrame({'Name':names, 'nbinarys':nbinarys})
masses_table = pd.merge(masses_table, db_nbinarys, on='Name').reset_index()

In [26]:
namedict = {0:' b', 1:' c', 2:' d', 3:' e', 4:' f', 5:' g', 6:' h', 7:' i'}

names = np.unique(np.array(masses_binary.hostname))
for name in names:
    masses_name = masses_table.query('Name == "{}"'.format(name))
    numbah = len(masses_name)
    pl_index = 0
    for index in np.arange(numbah) + masses_name.index[0]:
        masses_table.loc[index, 'Name'] += namedict[pl_index]  
        pl_index += 1
     
masses_binary['Name'] = masses_table['Name'].str.upper()

In [27]:
def round_to_1(x):
    if np.isnan(x):
        return 0.0001
    else:
        return round(x, -int(floor(log10(x))))

def round_sig(x, sig=2):
    if x == np.nan:
        return np.nan
    else:
        return round(x, sig-int(floor(log10(abs(x)))) - 1) 
round_sig_vec = np.vectorize(round_sig)

def round_to_reference(x, y):
    if x == np.nan or y == np.nan:
        return np.nan
    else:
        return round(x, -int(floor(log10(abs(y)))) + 1)
round_to_ref_vec = np.vectorize(round_to_reference)

## Make table for planet parameters.

In [29]:
# Round errors and parameters with significant figures.
masses_planet['mass_err_plus_r']  = round_sig_vec(np.array(masses_planet['mass_plus'] - masses_planet['mass_med']))
masses_planet['mass_err_minus_r'] = round_sig_vec(np.array(masses_planet['mass_med'] - masses_planet['mass_minus']))
masses_planet['mass_err_av']      = round_sig_vec(0.5*(masses_planet['mass_err_plus_r'] + masses_planet['mass_err_minus_r']))
masses_planet['mass_med_r']       = round_to_ref_vec(np.array(masses_planet['mass_med']), masses_planet['mass_err_av'])

masses_planet['axis_err_plus_r']  = round_sig_vec(np.array(masses_planet['axis_plus'] - masses_planet['axis_med']))
masses_planet['axis_err_minus_r'] = round_sig_vec(np.array(masses_planet['axis_med'] - masses_planet['axis_minus']))
masses_planet['axis_err_av']      = round_sig_vec(0.5*(masses_planet['axis_err_plus_r'] + masses_planet['axis_err_minus_r']))
masses_planet['axis_med_r']       = round_to_ref_vec(np.array(masses_planet['axis_med']), masses_planet['axis_err_av'])

masses_planet['per_err_plus_r']   = round_sig_vec(np.array(masses_planet['per_plus'] - masses_planet['per_med']))
masses_planet['per_err_minus_r']  = round_sig_vec(np.array(masses_planet['per_med'] - masses_planet['per_minus']))
masses_planet['per_err_av']       = round_sig_vec(0.5*(masses_planet['per_err_plus_r'] + masses_planet['per_err_minus_r']))
masses_planet['per_med_r']        = round_to_ref_vec(np.array(masses_planet['per_med']), masses_planet['per_err_av'])

masses_planet['e_err_plus_r']     = round_sig_vec(np.array(masses_planet['e_plus'] - masses_planet['e_med']))
masses_planet['e_err_minus_r']    = round_sig_vec(np.array(masses_planet['e_med'] - masses_planet['e_minus']))
masses_planet['e_err_av']         = round_sig_vec(0.5*(masses_planet['e_err_plus_r'] + masses_planet['e_err_minus_r']))
masses_planet['e_med_r']          = round_to_ref_vec(np.array(masses_planet['e_med']), masses_planet['e_err_av'])
'''
masses_planet['tc_err_plus_r']     = round_sig_vec(np.array(masses_planet['tc_plus'] - masses_planet['tc_med']))
masses_planet['tc_err_minus_r']    = round_sig_vec(np.array(masses_planet['tc_med'] - masses_planet['tc_minus']))
masses_planet['tc_err_av']         = round_sig_vec(0.5*(masses_planet['tc_err_plus_r'] + masses_planet['tc_err_minus_r']))
masses_planet['tc_med_r']          = round_to_ref_vec(np.array(masses_planet['tc_med']), masses_planet['tc_err_av'])

masses_planet['w_err_plus_r']     = round_sig_vec(np.array(masses_planet['w_plus'] - masses_planet['w_med']))
masses_planet['w_err_minus_r']    = round_sig_vec(np.array(masses_planet['w_med'] - masses_planet['w_minus']))
masses_planet['w_err_av']         = round_sig_vec(0.5*(masses_planet['w_err_plus_r'] + masses_planet['w_err_minus_r']))
masses_planet['w_med_r']          = round_to_ref_vec(np.array(masses_planet['w_med']), masses_planet['w_err_av'])
'''
tex_writer = open('legacy_tables/tex_tables/planet_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{llrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\caption{Planet Catalog}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'CPS Name & Lit. Name & \msini\ [\mjup] & a [AU] & $P$ [d] & $e$ \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endfirsthead' + ' \n')
tex_writer.write(r'\caption[]{Planet Catalog (Continued)}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write(r'CPS Name & Lit. Name & \msini\ [\mjup] & a [AU] & $P$ [d] & $e$ \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endhead' + ' \n')

for index, row in masses_planet.iterrows():
    # Customize names.
    namein = str(row['Name'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein

    # Make sure that uncertainties have the right number of significant figures, including final zeros.
    mass_str = str(row['mass_med_r'])
    mass_plus_str  = str(row['mass_err_plus_r'])
    mass_minus_str = str(row['mass_err_minus_r'])
    if len(mass_plus_str) < len(mass_minus_str) and (mass_plus_str[:2] == '0.' or mass_minus_str[:2] == '0.'):
        mass_plus_str += '0'
    elif len(mass_plus_str) > len(mass_minus_str) and (mass_plus_str[:2] == '0.' or mass_minus_str[:2] == '0.'):
        mass_minus_str += '0'    
    
    if mass_str[-2:] == '.0' and mass_plus_str[-2:] == '.0' and mass_minus_str[-2:] == '.0':
        mass_str = mass_str[-2:]
        mass_plus_str = mass_plus_str[-2:]
        mass_minus_str = mass_minus_str[-2:]
        
    axis_str = str(row['axis_med_r'])
    axis_plus_str  = str(row['axis_err_plus_r'])
    axis_minus_str = str(row['axis_err_minus_r'])
    if len(axis_plus_str) < len(axis_minus_str) and (axis_plus_str[:2] == '0.' or axis_minus_str[:2] == '0.'):
        axis_plus_str += '0'
    elif len(axis_plus_str) > len(axis_minus_str) and (axis_plus_str[:2] == '0.' or axis_minus_str[:2] == '0.'):
        axis_minus_str += '0' 
        
    if axis_str[-2:] == '.0' and axis_plus_str[-2:] == '.0' and axis_minus_str[-2:] == '.0':
        axis_str = axis_str[-2:]
        axis_plus_str = axis_plus_str[-2:]
        axis_minus_str = axis_minus_str[-2:]
        
    per_str = str(row['per_med_r'])
    per_plus_str  = str(row['per_err_plus_r'])
    per_minus_str = str(row['per_err_minus_r'])
    if len(per_plus_str) < len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_plus_str += '0'
    elif len(per_plus_str) > len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_minus_str += '0' 
        
    if per_str[-2:] == '.0' and per_plus_str[-2:] == '.0' and per_minus_str[-2:] == '.0':
        per_str = per_str[-2:]
        per_plus_str = per_plus_str[-2:]
        per_minus_str = per_minus_str[-2:]
        
    ecc_str = str(row['e_med_r'])
    ecc_plus_str  = str(row['e_err_plus_r'])
    ecc_minus_str = str(row['e_err_minus_r'])
    if len(ecc_plus_str) < len(ecc_minus_str) and (ecc_plus_str[:2] == '0.' or ecc_minus_str[:2] == '0.'):
        ecc_plus_str += '0'
    elif len(ecc_plus_str) > len(ecc_minus_str) and (ecc_plus_str[:2] == '0.' or ecc_minus_str[:2] == '0.'):
        ecc_minus_str += '0'
        
    massin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(mass_str, mass_plus_str, mass_minus_str) + r'$'    
    axisin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(axis_str, axis_plus_str, axis_minus_str) + r'$'    
    periin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(per_str, per_plus_str, per_minus_str) + r'$'
    eccein = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(ecc_str, ecc_plus_str, ecc_minus_str) + r'$'
    
    '''
    tcin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['tc_med_r'], 
                                                 row['tc_err_plus_r'], 
                                                 row['tc_err_minus_r']) + r'$'
    win = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['w_med_r'], 
                                                row['w_err_plus_r'], 
                                                row['w_err_minus_r']) + r'$'
    '''
    
    tex_writer.write('{0} & {1} & {2} & {3} & {4} & {5} \\\\ \n'.format(
                     namein, namein, massin, axisin, periin, eccein))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

# Make table for stellar parameters. 

In [9]:
# Specmatch parameters first. TO-DO: COMBINE SYNTHETIC/EMPIRICIAL RESULTS. MAKE COLUMN WITH SYN/EMP
stellar['teff_err_r'] = round_sig_vec(stellar['teff_err_c'])
stellar['teff_r'] = round_to_ref_vec(np.array(stellar['teff_c']), stellar['teff_err_r'])
stellar['fe_err_r'] = round_sig_vec(stellar['fe_err_c'])
stellar['fe_r'] = round_to_ref_vec(np.array(stellar['fe_c']), stellar['fe_err_r'])

# USE ISOCLASSIFY OUTPUTS FOR R AND LOGG, SINCE SPECMATCH-SYN AND -EMP EACH ONLY PRODUCE ONE. 
# FIX FIX FIX FIX FIX FIX FIX FIX FIX FIX FIX
stellar['logg_err_r'] = round_sig_vec(stellar['logg_err_c'])
stellar['logg_r'] = round_to_ref_vec(np.array(stellar['logg_c']), stellar['logg_err_r'])
stellar['radius_err_r'] = round_sig_vec(stellar['radius_err_c'])
stellar['radius_r'] = round_to_ref_vec(np.array(stellar['radius_c']), stellar['radius_err_r'])

# Now isoclassify.
stellar['mass_err_r'] = round_sig_vec(stellar['mass_err_c'])
stellar['mass_r'] = round_to_ref_vec(np.array(stellar['mass_c']), stellar['mass_err_r'])


tex_writer = open('legacy_tables/tex_tables/stellar_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{lrrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\caption{Stellar Catalog}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'Name & $T_{\textrm{eff}}$ [K] & [Fe/H] & log $g$ [log(cm s$^{-2}$)] & $R$ [\rsun] & $M$ [\msun] \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endfirsthead' + ' \n')
tex_writer.write(r'\caption[]{Stellar Catalog (Continued)}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write(r'Name & $T_{\textrm{eff}}$ [K] & [Fe/H] & log $g$ [log(cm s$^{-2}$)] & $R$ [\rsun] & $M$ [\msun] \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endhead' + ' \n')

for index, row in stellar.iterrows():
    # Customize names.
    namein = str(row['name'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein

    teff_str   = r'$' + '{0}\\pm {1}'.format(row['teff_r'], row['teff_err_r']) + r'$'    
    fe_str     = r'$' + '{0}\\pm {1}'.format(row['fe_r'], row['fe_err_r']) + r'$'    
    logg_str   = r'$' + '{0}\\pm {1}'.format(row['logg_r'], row['logg_err_r']) + r'$' 
    radius_str = r'$' + '{0}\\pm {1}'.format(row['radius_r'], row['radius_err_r']) + r'$'      
    mass_str   = r'$' + '{0}\\pm {1}'.format(row['mass_r'], row['mass_err_r']) + r'$'

    tex_writer.write('{0} & {1} & {2} & {3} & {4} & {5} \\\\ \n'.format(
                     namein, teff_str, fe_str, logg_str, radius_str, mass_str))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

## Make new candidate table.

In [28]:
masses_candidate = masses_planet.query('status == "C" or status == "J"').reset_index(drop=True)

# Round errors and parameters with significant figures.
masses_candidate['mass_err_plus_r']  = round_sig_vec(np.array(masses_candidate['mass_plus'] - masses_candidate['mass_med']))
masses_candidate['mass_err_minus_r'] = round_sig_vec(np.array(masses_candidate['mass_med'] - masses_candidate['mass_minus']))
masses_candidate['mass_err_av']      = round_sig_vec(0.5*(masses_candidate['mass_err_plus_r'] + masses_candidate['mass_err_minus_r']))
masses_candidate['mass_med_r']       = round_to_ref_vec(np.array(masses_candidate['mass_med']), masses_candidate['mass_err_av'])

masses_candidate['axis_err_plus_r']  = round_sig_vec(np.array(masses_candidate['axis_plus'] - masses_candidate['axis_med']))
masses_candidate['axis_err_minus_r'] = round_sig_vec(np.array(masses_candidate['axis_med'] - masses_candidate['axis_minus']))
masses_candidate['axis_err_av']      = round_sig_vec(0.5*(masses_candidate['axis_err_plus_r'] + masses_candidate['axis_err_minus_r']))
masses_candidate['axis_med_r']       = round_to_ref_vec(np.array(masses_candidate['axis_med']), masses_candidate['axis_err_av'])

masses_candidate['per_err_plus_r']   = round_sig_vec(np.array(masses_candidate['per_plus'] - masses_candidate['per_med']))
masses_candidate['per_err_minus_r']  = round_sig_vec(np.array(masses_candidate['per_med'] - masses_candidate['per_minus']))
masses_candidate['per_err_av']       = round_sig_vec(0.5*(masses_candidate['per_err_plus_r'] + masses_candidate['per_err_minus_r']))
masses_candidate['per_med_r']        = round_to_ref_vec(np.array(masses_candidate['per_med']), masses_candidate['per_err_av'])

masses_candidate['e_err_plus_r']     = round_sig_vec(np.array(masses_candidate['e_plus'] - masses_candidate['e_med']))
masses_candidate['e_err_minus_r']    = round_sig_vec(np.array(masses_candidate['e_med'] - masses_candidate['e_minus']))
masses_candidate['e_err_av']         = round_sig_vec(0.5*(masses_candidate['e_err_plus_r'] + masses_candidate['e_err_minus_r']))
masses_candidate['e_med_r']          = round_to_ref_vec(np.array(masses_candidate['e_med']), masses_candidate['e_err_av'])
'''
masses_candidate['tc_err_plus_r']     = round_sig_vec(np.array(masses_candidate['tc_plus'] - masses_candidate['tc_med']))
masses_candidate['tc_err_minus_r']    = round_sig_vec(np.array(masses_candidate['tc_med'] - masses_candidate['tc_minus']))
masses_candidate['tc_err_av']         = round_sig_vec(0.5*(masses_candidate['tc_err_plus_r'] + masses_candidate['tc_err_minus_r']))
masses_candidate['tc_med_r']          = round_to_ref_vec(np.array(masses_candidate['tc_med']), masses_candidate['tc_err_av'])

masses_candidate['w_err_plus_r']     = round_sig_vec(np.array(masses_candidate['w_plus'] - masses_candidate['w_med']))
masses_candidate['w_err_minus_r']    = round_sig_vec(np.array(masses_candidate['w_med'] - masses_candidate['w_minus']))
masses_candidate['w_err_av']         = round_sig_vec(0.5*(masses_candidate['w_err_plus_r'] + masses_candidate['w_err_minus_r']))
masses_candidate['w_med_r']          = round_to_ref_vec(np.array(masses_candidate['w_med']), masses_candidate['w_err_av'])
'''
tex_writer = open('legacy_tables/tex_tables/candidate_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{llrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\caption{Newly Discovered Planets and Substellar Companions}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'CPS Name & Lit. Name & \msini\ [\mjup] & a [AU] & $P$ [d] & $e$ \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')

for index, row in masses_candidate.iterrows():
    # Customize names.
    namein = str(row['Name'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein

    # Make sure that uncertainties have the right number of significant figures, including final zeros.
    mass_str = str(row['mass_med_r'])
    mass_plus_str  = str(row['mass_err_plus_r'])
    mass_minus_str = str(row['mass_err_minus_r'])
    if len(mass_plus_str) < len(mass_minus_str) and (mass_plus_str[:2] == '0.' or mass_minus_str[:2] == '0.'):
        mass_plus_str += '0'
    elif len(mass_plus_str) > len(mass_minus_str) and (mass_plus_str[:2] == '0.' or mass_minus_str[:2] == '0.'):
        mass_minus_str += '0'    
    
    if mass_str[-2:] == '.0' and mass_plus_str[-2:] == '.0' and mass_minus_str[-2:] == '.0':
        mass_str = mass_str[-2:]
        mass_plus_str = mass_plus_str[-2:]
        mass_minus_str = mass_minus_str[-2:]
        
    axis_str = str(row['axis_med_r'])
    axis_plus_str  = str(row['axis_err_plus_r'])
    axis_minus_str = str(row['axis_err_minus_r'])
    if len(axis_plus_str) < len(axis_minus_str) and (axis_plus_str[:2] == '0.' or axis_minus_str[:2] == '0.'):
        axis_plus_str += '0'
    elif len(axis_plus_str) > len(axis_minus_str) and (axis_plus_str[:2] == '0.' or axis_minus_str[:2] == '0.'):
        axis_minus_str += '0' 
        
    if axis_str[-2:] == '.0' and axis_plus_str[-2:] == '.0' and axis_minus_str[-2:] == '.0':
        axis_str = axis_str[-2:]
        axis_plus_str = axis_plus_str[-2:]
        axis_minus_str = axis_minus_str[-2:]
        
    per_str = str(row['per_med_r'])
    per_plus_str  = str(row['per_err_plus_r'])
    per_minus_str = str(row['per_err_minus_r'])
    if len(per_plus_str) < len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_plus_str += '0'
    elif len(per_plus_str) > len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_minus_str += '0' 
        
    if per_str[-2:] == '.0' and per_plus_str[-2:] == '.0' and per_minus_str[-2:] == '.0':
        per_str = per_str[-2:]
        per_plus_str = per_plus_str[-2:]
        per_minus_str = per_minus_str[-2:]
        
    ecc_str = str(row['e_med_r'])
    ecc_plus_str  = str(row['e_err_plus_r'])
    ecc_minus_str = str(row['e_err_minus_r'])
    if len(ecc_plus_str) < len(ecc_minus_str) and (ecc_plus_str[:2] == '0.' or ecc_minus_str[:2] == '0.'):
        ecc_plus_str += '0'
    elif len(ecc_plus_str) > len(ecc_minus_str) and (ecc_plus_str[:2] == '0.' or ecc_minus_str[:2] == '0.'):
        ecc_minus_str += '0' 
        
    massin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(mass_str, mass_plus_str, mass_minus_str) + r'$'    
    axisin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(axis_str, axis_plus_str, axis_minus_str) + r'$'    
    periin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(per_str, per_plus_str, per_minus_str) + r'$'
    eccein = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(ecc_str, ecc_plus_str, ecc_minus_str) + r'$'
    '''
    tcin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['tc_med_r'], 
                                                 row['tc_err_plus_r'], 
                                                 row['tc_err_minus_r']) + r'$'
    win = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['w_med_r'], 
                                                row['w_err_plus_r'], 
                                                row['w_err_minus_r']) + r'$'
    '''
    tex_writer.write('{0} & {1} & {2} & {3} & {4} & {5} \\\\ \n'.format(
                     namein, namein, massin, axisin, periin, eccein))    
    #tex_writer.write('{0} & {1} & {2} & {3} & {4} \\\\ \n'.format(
    #                 namein, massin, axisin, periin, eccein))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

## Make false-positive table.

In [18]:
false_positive['per_err_plus_r']   = round_sig_vec(np.array(false_positive['per_plus'] - false_positive['per_med']))
false_positive['per_err_minus_r']  = round_sig_vec(np.array(false_positive['per_med'] - false_positive['per_minus']))
false_positive['per_err_av']       = round_sig_vec(0.5*(false_positive['per_err_plus_r'] + false_positive['per_err_minus_r']))
false_positive['per_med_r']        = round_to_ref_vec(np.array(false_positive['per_med']), false_positive['per_err_av'])

false_positive['k_err_plus_r']   = round_sig_vec(np.array(false_positive['k_plus'] - false_positive['k_med']))
false_positive['k_err_minus_r']  = round_sig_vec(np.array(false_positive['k_med'] - false_positive['k_minus']))
false_positive['k_err_av']       = round_sig_vec(0.5*(false_positive['k_err_plus_r'] + false_positive['k_err_minus_r']))
false_positive['k_med_r']        = round_to_ref_vec(np.array(false_positive['k_med']), false_positive['k_err_av'])

false_positive['k_r'] = round_sig_vec(np.array(false_positive['k_med']))

tex_writer = open('legacy_tables/tex_tables/false_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{llrrr}')
tex_writer.write('\n')
tex_writer.write(r'\caption{False Positives}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'CPS Name & Lit. Name & $P$ [d] & $K$ [m s$^{-1}$] & Cause \\' + ' \n')
tex_writer.write(r'\endfirsthead' + ' \n')
tex_writer.write(r'\caption[]{False Positives (Continued)}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write(r'CPS Name & Lit. Name & $P$ [d] & $K$ [m s$^{-1}$] & Cause \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endhead' + ' \n')

for index, row in false_positive.iterrows():
    # Customize names.
    namein = str(row['hostname'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein[:]
        
    per_str = str(row['per_med_r'])
    per_plus_str  = str(row['per_err_plus_r'])
    per_minus_str = str(row['per_err_minus_r'])
    if len(per_plus_str) < len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_plus_str += '0'
    elif len(per_plus_str) > len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_minus_str += '0' 
        
    k_str = str(row['k_med_r'])
    k_plus_str  = str(row['k_err_plus_r'])
    k_minus_str = str(row['k_err_minus_r'])
    if len(k_plus_str) < len(k_minus_str) and (k_plus_str[:2] == '0.' or k_minus_str[:2] == '0.'):
        k_plus_str += '0'
    elif len(k_plus_str) > len(k_minus_str) and (k_plus_str[:2] == '0.' or k_minus_str[:2] == '0.'):
        k_minus_str += '0' 

    periin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(per_str, per_plus_str, per_minus_str) + r'$'
    kin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(k_str, k_plus_str, k_minus_str) + r'$'
    reason = str(row['status'])

    tex_writer.write('{0} & {1} & {2} & {3} & {4} \\\\ \n'.format(
                     namein, namein, periin, kin, reason))    
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

## Make stellar/substellar binary table.

In [43]:
masses_binary = masses_binary.fillna(10.5)
masses_binary = masses_binary.query('mass_med > 0 and mass_minus > 0 and mass > 0 and \
                                     k_med > 0 and k_minus > 0 and k > 0 and \
                                     axis_med > 0 and axis_minus > 0 and axis > 0 and \
                                     per_med > 0 and per_minus > 0 and per > 0 and \
                                     hostname != "3130-1591-1"').reset_index()

In [44]:
masses_binary[['hostname', 'axis', 'axis_minus', 'axis_med', 'axis_plus']]

,hostname,axis,axis_minus,axis_med,axis_plus
0,10790,1.541407,1.516416,1.541341,1.565451
1,112914,1.382148,1.364059,1.382217,1.399670
2,116321,7.726527,8.735236,16.282146,33.479137
3,120136,22.329765,14.316500,18.229948,35.392300
4,142229,13.628901,16.093541,24.469100,37.095153
5,142267,0.490494,0.483678,0.490507,0.497126
6,157338,17.543707,17.277640,24.328158,33.414174
7,159062,17.969802,32.506792,57.096979,87.380092
8,16160,15.790276,15.990030,16.307961,16.634160
9,161797,15.053919,15.088640,16.180753,17.386471


In [45]:
masses_binary['mass_err_plus_r']  = round_sig_vec(np.abs(np.array(masses_binary['mass_plus'] - masses_binary['mass_med'])))
masses_binary['mass_err_minus_r'] = round_sig_vec(np.abs(np.array(masses_binary['mass_med'] - masses_binary['mass_minus'])))
masses_binary['mass_err_av']      = round_sig_vec(0.5*(masses_binary['mass_err_plus_r'] + masses_binary['mass_err_minus_r']))
masses_binary['mass_med_r']       = round_to_ref_vec(np.array(masses_binary['mass_med']), masses_binary['mass_err_av'])

masses_binary['axis_err_plus_r']  = round_sig_vec(np.abs(np.array(masses_binary['axis_plus'] - masses_binary['axis_med'])))
masses_binary['axis_err_minus_r'] = round_sig_vec(np.abs(np.array(masses_binary['axis_med'] - masses_binary['axis_minus'])))
masses_binary['axis_err_av']      = round_sig_vec(0.5*(masses_binary['axis_err_plus_r'] + masses_binary['axis_err_minus_r']))
masses_binary['axis_med_r']       = round_to_ref_vec(np.array(masses_binary['axis_med']), masses_binary['axis_err_av'])

masses_binary['per_err_plus_r']   = round_sig_vec(np.abs(np.array(masses_binary['per_plus'] - masses_binary['per_med'])))
masses_binary['per_err_minus_r']  = round_sig_vec(np.abs(np.array(masses_binary['per_med'] - masses_binary['per_minus'])))
masses_binary['per_err_av']       = round_sig_vec(0.5*(masses_binary['per_err_plus_r'] + masses_binary['per_err_minus_r']))
masses_binary['per_med_r']        = round_to_ref_vec(np.array(masses_binary['per_med']), masses_binary['per_err_av'])

masses_binary['e_err_plus_r']     = round_sig_vec(np.abs(np.array(masses_binary['e_plus'] - masses_binary['e_med'])))
masses_binary['e_err_minus_r']    = round_sig_vec(np.abs(np.array(masses_binary['e_med'] - masses_binary['e_minus'])))
masses_binary['e_err_av']         = round_sig_vec(0.5*(masses_binary['e_err_plus_r'] + masses_binary['e_err_minus_r']))
masses_binary['e_med_r']          = round_to_ref_vec(np.array(masses_binary['e_med']), masses_binary['e_err_av'])
'''
masses_binary['tc_err_plus_r']     = round_sig_vec(np.array(masses_binary['tc_plus'] - masses_binary['tc_med']))
masses_binary['tc_err_minus_r']    = round_sig_vec(np.array(masses_binary['tc_med'] - masses_binary['tc_minus']))
masses_binary['tc_err_av']         = round_sig_vec(0.5*(masses_binary['tc_err_plus_r'] + masses_binary['tc_err_minus_r']))
masses_binary['tc_med_r']          = round_to_ref_vec(np.array(masses_binary['tc_med']), masses_binary['tc_err_av'])

masses_binary['w_err_plus_r']     = round_sig_vec(np.array(masses_binary['w_plus'] - masses_binary['w_med']))
masses_binary['w_err_minus_r']    = round_sig_vec(np.array(masses_binary['w_med'] - masses_binary['w_minus']))
masses_binary['w_err_av']         = round_sig_vec(0.5*(masses_binary['w_err_plus_r'] + masses_binary['w_err_minus_r']))
masses_binary['w_med_r']          = round_to_ref_vec(np.array(masses_binary['w_med']), masses_binary['w_err_av'])
'''
tex_writer = open('legacy_tables/tex_tables/binary_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{lrrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\caption{Binary and Substellar Catalog}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'Name & \msini\ [\mjup] & a [AU] & $P$ [d] & $e$ \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endfirsthead' + ' \n')
tex_writer.write(r'\caption[]{Binary and Substellar Catalog (Continued)}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write(r'Name & \msini\ [\mjup] & a [AU] & $P$ [d] & $e$ \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endhead' + ' \n')

for index, row in masses_binary.iterrows():
    # Customize names.
    namein = str(row['hostname'])
    if namein[0] == 'h' or namein[0] == 'H':
        namein = namein[:3].upper() + ' ' + namein[3:].upper()
    elif namein[0] == 'g' or namein[0] == 'G':
        namein = namein[:2].upper() + ' ' + namein[2:].upper()
    elif namein[0] == 'b' or namein[0] == 'B':
        namein = namein[:2].upper() + namein[2:].upper()
    else:
        namein = 'HD ' + namein

    # Make sure that uncertainties have the right number of significant figures, including final zeros.
    mass_str = str(row['mass_med_r'])
    mass_plus_str  = str(row['mass_err_plus_r'])
    mass_minus_str = str(row['mass_err_minus_r'])
    if len(mass_plus_str) < len(mass_minus_str) and (mass_plus_str[:2] == '0.' or mass_minus_str[:2] == '0.'):
        mass_plus_str += '0'
    elif len(mass_plus_str) > len(mass_minus_str) and (mass_plus_str[:2] == '0.' or mass_minus_str[:2] == '0.'):
        mass_minus_str += '0'    
        
    axis_str = str(row['axis_med_r'])
    axis_plus_str  = str(row['axis_err_plus_r'])
    axis_minus_str = str(row['axis_err_minus_r'])
    if len(axis_plus_str) < len(axis_minus_str) and (axis_plus_str[:2] == '0.' or axis_minus_str[:2] == '0.'):
        axis_plus_str += '0'
    elif len(axis_plus_str) > len(axis_minus_str) and (axis_plus_str[:2] == '0.' or axis_minus_str[:2] == '0.'):
        axis_minus_str += '0' 
        
    per_str = str(row['per_med_r'])
    per_plus_str  = str(row['per_err_plus_r'])
    per_minus_str = str(row['per_err_minus_r'])
    if len(per_plus_str) < len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_plus_str += '0'
    elif len(per_plus_str) > len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_minus_str += '0' 
        
    ecc_str = str(row['e_med_r'])
    ecc_plus_str  = str(row['e_err_plus_r'])
    ecc_minus_str = str(row['e_err_minus_r'])
    if len(ecc_plus_str) < len(ecc_minus_str) and (ecc_plus_str[:2] == '0.' or ecc_minus_str[:2] == '0.'):
        ecc_plus_str += '0'
    elif len(ecc_plus_str) > len(ecc_minus_str) and (ecc_plus_str[:2] == '0.' or ecc_minus_str[:2] == '0.'):
        ecc_minus_str += '0' 
        
    massin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(mass_str, mass_plus_str, mass_minus_str) + r'$'    
    axisin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(axis_str, axis_plus_str, axis_minus_str) + r'$'    
    periin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(per_str, per_plus_str, per_minus_str) + r'$'
    eccein = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(ecc_str, ecc_plus_str, ecc_minus_str) + r'$'
    '''
    tcin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['tc_med_r'], 
                                                 row['tc_err_plus_r'], 
                                                 row['tc_err_minus_r']) + r'$'
    win = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['w_med_r'], 
                                                row['w_err_plus_r'], 
                                                row['w_err_minus_r']) + r'$'
    '''
    
    tex_writer.write('{0} & {1} & {2} & {3} & {4} \\\\ \n'.format(
                     namein, massin, axisin, periin, eccein))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

## Make table for trend and curvature terms.

In [71]:
trends_curv = trends.query('curv == curv')
#pdb.set_trace()

# Round errors and parameters with significant figures.
trends['dvdt_err_plus_r']  = round_sig_vec(np.array(trends['dvdt_plus'] - trends['dvdt_med']))
trends['dvdt_err_minus_r'] = round_sig_vec(np.array(trends['dvdt_med'] - trends['dvdt_minus']))
trends['dvdt_err_av']      = round_sig_vec(0.5*(trends['dvdt_err_plus_r'] + trends['dvdt_err_minus_r']))
trends['dvdt_med_r']       = round_to_ref_vec(np.array(trends['dvdt_med']), trends['dvdt_err_av'])

# Combine curvature and non-curvature databases.
trends_curv['curv_err_plus_r']  = round_sig_vec(np.array(trends_curv['curv_plus'] - trends_curv['curv_med']))
trends_curv['curv_err_minus_r'] = round_sig_vec(np.array(trends_curv['curv_med'] - trends_curv['curv_minus']))
trends_curv['curv_err_av']      = round_sig_vec(0.5*(trends_curv['curv_err_plus_r'] + trends_curv['curv_err_minus_r']))
trends_curv['curv_med_r']       = round_to_ref_vec(np.array(trends_curv['curv_med']), trends_curv['curv_err_av'])

#trends = trends.merge(trends_curv, on='hostname', suffixes=('', '_curv'))
#trends = trends.update(trends_curv)
#pdb.set_trace()

tex_writer = open('legacy_tables/tex_tables/trends_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{lrrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\caption{Long Term Trends}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'Name & $\dot \gamma$ [m/s/yr] & $\ddot \gamma$ [m/s$^2$/yr] \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endfirsthead' + ' \n')
tex_writer.write(r'\caption[]{Long Term Trends (Continued)}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write(r'Name & $\dot \gamma$ [m/s/yr] & $\ddot \gamma$ [m/s$^2$/yr] \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endhead' + ' \n')
#tex_writer.write(r'Name & dvdt & curvature \\' + ' \n')

for index, row in trends.iterrows():
    # Customize names.
    basename = str(row['hostname'])
    namein = str(row['hostname'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:].upper()
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:].upper()
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:].upper()
    else:
        namein = 'HD ' + namein.upper()

    dvdtin = r'$' + '{0}\\pm {1}'.format(row['dvdt_med_r'], 
                                         row['dvdt_err_av']) + r'$'
    
    if basename in trends_curv['hostname'].tolist():
        row = trends_curv.query('hostname == "{}"'.format(basename))
        meanstr = str(row['curv_med_r'].to_list()[0])
        errstr  = str(row['curv_err_av'].to_list()[0])

        if 'e' in meanstr:
            meanbase, meanexp = str.split(meanstr, sep='e')
            errbase, errexp = str.split(errstr, sep='e')
            #print(meanbase, meanexp)
            curvin = r'$' + '{0}*10^{1}\\pm {2}*10^{3}'.format(meanbase, 
                                                               '{-' + meanexp[-1] + '}', 
                                                                 errbase, 
                                                               '{-' + errexp[-1]) + '}' + r'$'
        else:
            curvin = r'$' + '{0}\\pm {1}'.format(meanstr, errstr) + r'$'    
        print(curvin)
    else:
        curvin = '0'

    
    tex_writer.write('{0} & {1} & {2} \\\\ \n'.format(namein, dvdtin, curvin))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

$0.00036\pm 0.00012$
$1.89*10^{-6}\pm 1.2*10^{-7}$
$-6.05*10^{-6}\pm 6.9*10^{-7}$
$2.25*10^{-6}\pm 3*10^{-7}$
$-9.3*10^{-7}\pm 1.1*10^{-7}$
$-4.27*10^{-6}\pm 1.1*10^{-7}$
$-9.03*10^{-6}\pm 8.6*10^{-7}$
$-3.23*10^{-7}\pm 5.6*10^{-8}$
$-4.32*10^{-7}\pm 6*10^{-8}$
$-5.2*10^{-7}\pm 1.5*10^{-7}$
$-8.76*10^{-7}\pm 9.6*10^{-8}$
$-4.09*10^{-7}\pm 6*10^{-8}$
$-6*10^{-7}\pm 1.7*10^{-7}$
$-4.41*10^{-6}\pm 2.4*10^{-7}$
$-4.61*10^{-6}\pm 4.3*10^{-7}$
$-2.21*10^{-5}\pm 3.8*10^{-6}$
$-2.96*10^{-5}\pm 5.5*10^{-6}$
$-4.3*10^{-6}\pm 1.8*10^{-7}$
$-3.17*10^{-6}\pm 3.9*10^{-7}$


In [11]:
data.columns

Index(['Unnamed: 0', 'JD_UTC', 'OBSNM', 'Prot', 'S-value', 'S/N', 'SVAL',
       'age', 'bc', 'comment', 'cts', 'date', 'date_sval', 'day', 'day_sval',
       'decker', 'dewar', 'errvel', 'gain', 'hjd_utc', 'hjd_utc_sval', 'inst',
       'iodnm', 'jd', 'jd_sval', 'jitter', 'kjd', 'kjd_sval', 'mdchi', 'mdpar',
       'mdvel', 'med_all', 'mnpar', 'mnvel', 'name', 'nchunk', 'obnm', 'obs',
       'phase', 'psfpix', 'psfsig', 'rphk', 'sn', 'sp1', 'sp2', 'spst', 'star',
       'sval', 'tel', 'z'],
      dtype='object')

In [22]:
data_select = data[['name', 'tel', 'obnm', 'jd', 'mnvel', 'errvel', 'sval']]

In [23]:
data_select.query('name == "95128"')

,name,tel,obnm,jd,mnvel,errvel,sval
87809,95128,j,rj67.734,2.454930e+06,-2.252112,1.364153,0.159
87810,95128,j,rj68.131,2.454955e+06,-1.052739,1.189354,0.159
87811,95128,j,rj69.122,2.454964e+06,8.094488,1.353151,0.160
87812,95128,j,rj70.154,2.454984e+06,11.618073,1.592626,0.160
87813,95128,j,rj70.437,2.454985e+06,8.088678,1.386373,0.153
...,...,...,...,...,...,...,...
88639,95128,lick,ri58.86,2.455229e+06,56.444183,7.413153,NaN
88640,95128,lick,ri58.87,2.455229e+06,54.623985,7.084578,NaN
88641,95128,lick,ri62.93,2.455313e+06,46.871433,4.576725,NaN
88642,95128,lick,ri62.94,2.455313e+06,52.812397,4.210793,NaN


In [39]:
data_sample = data_select.query('name == "156668"')[:13]

In [42]:
tex_writer = open('legacy_tables/tex_tables/data_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{lrrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\caption{Sample of RV Data}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'Name & Inst. & BJD - 2455000 & $RV$ [m/s] & $\sigma_{RV}$ [m/s] & S-Value \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endfirsthead' + ' \n')
tex_writer.write(r'\caption[]{Sample of RV Data (Continued)}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write(r'Name & Inst. & BJD - 2455000 & $RV$ [m/s] & $\sigma_{RV}$ [m/s] & S-Value \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endhead' + ' \n')

for index, row in data_sample.iterrows():
    # Customize names.
    namein = str(row['name'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:].upper()
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:].upper()
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:].upper()
    elif namein[0] == 's':
        namein = namein.upper()
    else:
        namein = 'HD ' + namein.upper()

    telstr    = row['tel']
    #obsstr    = 
    jdstr     = str(round(row['jd'] - 2455000, 4))
    mnvelstr  = str(round(row['mnvel'], 2))
    errvelstr = str(round(row['errvel'], 2))
    svalstr   = str(round(row['sval'], 3))
    
    tex_writer.write('{0} & {1} & {2} & {3} & {4} & {5} \\\\ \n'.format(
                     namein, telstr, jdstr, mnvelstr, errvelstr, svalstr))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

In [28]:
len(data)

103991